In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("MLlib lab") \
    .getOrCreate()

sc = spark.sparkContext

24/04/26 16:05:01 WARN Utils: Your hostname, mac.local resolves to a loopback address: 127.0.0.1; using 10.132.197.78 instead (on interface en0)
24/04/26 16:05:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/26 16:05:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import Row

# Load Dataset

In [3]:
df_traindata = spark.read.csv("../data/traindata.csv", header=True, inferSchema=True)

In [4]:
df_traindata.printSchema()

root
 |-- separation real: double (nullable = true)
 |-- propensity real: double (nullable = true)
 |-- length real: double (nullable = true)
 |-- PredSS_r1_-4 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_-3 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_-2 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_-1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1 {H,E,C}: string (nullable = true)
 |-- PredSS_r1_1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_2 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_3 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_4 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-4 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-3 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-2 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2 {H,E,C}: string (nullable = true)
 |-- PredSS_r2_1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_2 {H,E,C,X}: string (nullable = true)
 |-- PredS

In [5]:
df_testdata = spark.read.csv("../data/testdata.csv", header=True, inferSchema=True)

In [6]:
df_testdata.printSchema()

root
 |-- separation real: double (nullable = true)
 |-- propensity real: double (nullable = true)
 |-- length real: double (nullable = true)
 |-- PredSS_r1_-4 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_-3 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_-2 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_-1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1 {H,E,C}: string (nullable = true)
 |-- PredSS_r1_1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_2 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_3 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r1_4 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-4 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-3 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-2 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_-1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2 {H,E,C}: string (nullable = true)
 |-- PredSS_r2_1 {H,E,C,X}: string (nullable = true)
 |-- PredSS_r2_2 {H,E,C,X}: string (nullable = true)
 |-- PredS

# Converts the values of class attribute with StringIndexer

In [7]:
string_col = []
float_col = []
for i in df_traindata.columns:
    if "real" not in i:
        string_col.append(i)
    else:
        float_col.append(i)

In [8]:
from pyspark.ml.feature import StringIndexer
df_traindata_p1 = df_traindata
df_testdata_p1 = df_testdata
for i in string_col:
    string_indexer = StringIndexer(inputCol=i, outputCol=i+"_indexed")
    df_traindata_p1 = string_indexer.fit(df_traindata).transform(df_traindata_p1)  
    df_testdata_p1 = string_indexer.fit(df_traindata).transform(df_testdata_p1) 
    df_traindata_p1 = df_traindata_p1.drop(i)
    df_testdata_p1 = df_testdata_p1.drop(i)

24/04/26 16:05:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
df_traindata_p1.printSchema()

root
 |-- separation real: double (nullable = true)
 |-- propensity real: double (nullable = true)
 |-- length real: double (nullable = true)
 |-- PredSS_freq_central_H real: double (nullable = true)
 |-- PredSS_freq_central_E real: double (nullable = true)
 |-- PredSS_freq_central_C real: double (nullable = true)
 |-- PredCN_freq_central_0 real: double (nullable = true)
 |-- PredCN_freq_central_1 real: double (nullable = true)
 |-- PredCN_freq_central_2 real: double (nullable = true)
 |-- PredCN_freq_central_3 real: double (nullable = true)
 |-- PredCN_freq_central_4 real: double (nullable = true)
 |-- PredRCH_freq_central_0 real: double (nullable = true)
 |-- PredRCH_freq_central_1 real: double (nullable = true)
 |-- PredRCH_freq_central_2 real: double (nullable = true)
 |-- PredRCH_freq_central_3 real: double (nullable = true)
 |-- PredRCH_freq_central_4 real: double (nullable = true)
 |-- PredSA_freq_central_0 real: double (nullable = true)
 |-- PredSA_freq_central_1 real: double (

In [10]:
df_testdata_p1.printSchema()

root
 |-- separation real: double (nullable = true)
 |-- propensity real: double (nullable = true)
 |-- length real: double (nullable = true)
 |-- PredSS_freq_central_H real: double (nullable = true)
 |-- PredSS_freq_central_E real: double (nullable = true)
 |-- PredSS_freq_central_C real: double (nullable = true)
 |-- PredCN_freq_central_0 real: double (nullable = true)
 |-- PredCN_freq_central_1 real: double (nullable = true)
 |-- PredCN_freq_central_2 real: double (nullable = true)
 |-- PredCN_freq_central_3 real: double (nullable = true)
 |-- PredCN_freq_central_4 real: double (nullable = true)
 |-- PredRCH_freq_central_0 real: double (nullable = true)
 |-- PredRCH_freq_central_1 real: double (nullable = true)
 |-- PredRCH_freq_central_2 real: double (nullable = true)
 |-- PredRCH_freq_central_3 real: double (nullable = true)
 |-- PredRCH_freq_central_4 real: double (nullable = true)
 |-- PredSA_freq_central_0 real: double (nullable = true)
 |-- PredSA_freq_central_1 real: double (